## III. Your first Rydberg Hamiltonian

These waveforms and atomic positions are, so far, just general mathematical functions. To wrap up this lesson, let's package these mathematical functions into physical parameters of a neutral-atom system via the Rydberg Hamiltonian defined in the beginning of this notebook. The simplest way to turn the mathematical functions into the different detuning, Rabi amplitude, phase, and atom position choices for a system we want to emulate or study with Aquila via the `rydberg_h` method. Here is what the basic syntax looks like:

In [1]:
from bloqade import rydberg_h, piecewise_linear, piecewise_constant, waveform, cast
from bloqade.atom_arrangement import ListOfLocations, Lieb, Square, Chain, Honeycomb, Kagome, Triangular, Rectangular
from bokeh.io import output_notebook # to plot "show()" on the notebook, without opening a browser
import os
import matplotlib.pyplot as plt
import numpy as np

output_notebook()

C:\Users\fabio\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Loading BokehJS ...

In [2]:
#geometry
rng = np.random.default_rng(1234)
atom_pos=Square(5, lattice_spacing=4).apply_defect_density(0.4, rng=rng).remove_vacant_sites()

#dynamics
deadtime=0.15
durations = [deadtime]
steps=25
tstep = 3.7/steps
for x in np.linspace(0, 4-2*deadtime, steps):
    durations.append(tstep)
durations.append(deadtime)

omega_MHz = [0.0]
omegamax = 2.5
for x in np.linspace(deadtime, 4-deadtime+tstep, steps+1):
    y = np.exp(-(x-3.8)/0.2)
    if (y>omegamax):
        y=omegamax
    omega_MHz.append(y)
omega_MHz.append(0.0)
Omega = piecewise_linear(durations,[x*2*np.pi for x in omega_MHz])


deltamax=15
deltamin=-20
delt=[deltamin]
for x in np.linspace(deadtime, 4-deadtime+tstep, steps+1):
    y = deltamax - ((deltamax-deltamin)/(np.exp((x-2)/0.25)+1))
    delt.append(y)
delt.append(deltamax)
Delta = piecewise_linear(durations,[z for z in delt])

#Delta = piecewise_linear(durations,[x*2*np.pi for x in delta_MHz])



#create Hamiltonian
program = rydberg_h(atom_pos, detuning= Delta, amplitude=Omega, phase=None)

program.parse_register().show()
program.parse_sequence().show()

Given a Hamiltonian describing a full program, we might as well run it to see what it does! Let's try first emulating the calculation.

In [ ]:
output = program.bloqade.python().run(shots=50, interaction_picture=True)

The simplest information we can extract from this, are the one hundred bit strings (from our 100 shots) that resulted from the calculation. The information lies inside the `report()` method

In [ ]:
output.report().bitstrings()

With these bit strings, we can measure observables, compute statistics, and do our general analysis. But they are a bit clumsy. We can get more information by calling counts, which immediatelly return us the number of times each bit string appeared as result of our calculation:

In [ ]:
output.report().counts()

Another option is to extract the meanvalue of some observables. For example, measuring $Z_i$ for site $i$, we can compute the mean Rydberg-state occupation density accross the system 

In [ ]:
output.report().rydberg_densities()

For convenience, basic data analysis and visualization can be obtained via Bloqade with a single show command on the report

In [ ]:
output.report().show()